## FCS Lab 2 Submission Report

* Name of Student(s): Leong Yun Qin Melody
* Student ID(s): 1004489

In [2]:
!pip3 install requests

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [1]:
import base64
import requests
def XOR(a, b):
    r = b""
    for x, y in zip(a, b):
        r += (x ^ y).to_bytes(1, "big")
    return r
class Client:
    def __init__(self, endpoint, uid):
        self.endpoint = endpoint
        self.uid = str(uid).lower().strip()
    def post(self, url, data=None):
        r = requests.post(url, json=data).json()
        if not r["success"]:
            print("Warning: something might be wrong with the server")
            print("If you don't think is your mistake, please report it!")
        return r
    def get_story_cipher(self):
        url = self.endpoint+"/story"
        return requests.get(url).json()
    def post_story_plaintext(self, solution):
        url = self.endpoint+"/story"
        solution = str(solution).lower().strip()
        data = {"solution":solution}
        return self.post(url, data)
    def get_score_msg_cipher(self):
        url = self.endpoint+"/score"
        data = {"request":"get_msg", "id":self.uid}
        return self.post(url, data)
    def submit_score_msg_cipher(self, cipher_base64):
        url = self.endpoint+"/score"
        data = {"request":"decrypt_msg", "id":self.uid, "cipher": cipher_base64}
        return self.post(url, data)
    def base64_encode_bytes(self, byte_array):
        return str(base64.b64encode(byte_array))[2:-1]
    def base64_decode_bytes(self, base64_string):
        return base64.b64decode(base64_string)

In [2]:
endpoint = "http://35.197.130.121/"
uid = "1004489"

client = Client(endpoint, uid)

## Part I: Story - Substitution Cipher

1. GET the cipher for the story
2. Crack this with frequency analysis
3. POST it back to the server to check (example is provided below)

If the response contains `'solution_correct': 'correct'`, then your decryption is correct. Otherwise, a distance will be provided to let you know how far off you are. If you are off by a tiny bit (say, 1 or 2), you can check things like line-ending, extra space at start/end etc. The verification is not case sensitive.

In [3]:
story_cipher = client.get_story_cipher()["cipher"]
print("story_cipher:", story_cipher[:50], "...")
#write to txt 

story_cipher: MXQJ YI IOCFXEWUQH. VEH Q BEEEEEDW, BEEEDW JYCU Y  ...


In [6]:
#decrypt with frequency analysis
#count number of each type of letters (take the proportion)
def count_freq_letters(story_cipher):
    #dictionary to map out letter to freq count
    letter = {"A":0, "B":0, "C":0, "D":0,"E":0, "F":0,
                "G":0, "H":0, "I":0,"J":0,"K":0,"L":0,"M":0,
                "N":0,"P":0,"Q":0, "R":0,"S":0,"T":0,"U":0,"V":0, "W":0,"X":0,"Y":0,"Z":0}
    #loop through the string 
    for alphabet in story_cipher:
        if alphabet in letter.keys():
            letter[alphabet] += 1

    #find the letter with largest count
    Counter = 0
    for char, freq_count in letter.items():
        #compare and store
        if freq_count > Counter:
            max_cipher_character = char
            Counter = freq_count
    #ensure the dict is correct 
    print(letter)
    return f"The letter appearing the most in the cipher text is {max_cipher_character}"

print(count_freq_letters(story_cipher))

#REPLACE the letter U with e       

{'A': 20, 'B': 102, 'C': 70, 'D': 205, 'E': 206, 'F': 43, 'G': 0, 'H': 131, 'I': 198, 'J': 263, 'K': 51, 'L': 29, 'M': 50, 'N': 3, 'P': 1, 'Q': 213, 'R': 35, 'S': 61, 'T': 98, 'U': 305, 'V': 50, 'W': 71, 'X': 160, 'Y': 229, 'Z': 3}
The letter appearing the most in the cipher text is U


In [ ]:
# replace the letter U with e
def replace_first_function(story_cipher):
    #make sure of original text 
    print(story_cipher)
    story_cipher.replace("U","e")
    return story_cipher
print(replace_first_function(story_cipher))
    

In [7]:
# example posting a string back to the server


client.post_story_plaintext("random")
# a distance is provided for you to check how close you are

<class 'str'>


{'distance': '3353',
 'hint': 'it is a substitution cipher; it is obvious when correct',
 'solution_correct': 'wrong',
 'success': True}

In [4]:
# you can also load solution from a text file
with open("./solution.txt", "r") as file:
    PART_1_SOLUTION = file.read()
part_1_result = client.post_story_plaintext(PART_1_SOLUTION)
print(part_1_result)
assert part_1_result["solution_correct"] == "correct"

FileNotFoundError: [Errno 2] No such file or directory: './solution.txt'

## Part II: Changing the Score Message - OTP

In [ ]:
response = client.get_score_msg_cipher()
print(response)

In [ ]:
cipher = client.base64_decode_bytes(response["cipher"])
print(cipher)

In [ ]:
encoded_cipher = client.base64_encode_bytes(cipher)
client.submit_score_msg_cipher(encoded_cipher)

In [ ]:
new_cipher = hax()
encoded_new_cipher = client.base64_encode_bytes(new_cipher)
client.submit_score_msg_cipher(encoded_new_cipher)